In [1]:
import numpy as np
from fsl.data.image import Image
from fsl.utils.image.resample import resampleToPixdims
from fsl.utils.image.roi import roi

In [2]:
path = "23679812_rsfmri_original"

In [3]:
img = Image(path).data

In [4]:
print(img.shape)

(91, 109, 91, 25)


In [5]:
img0 = img[:,:,:,0]

In [6]:
img0.shape

(91, 109, 91)

In [8]:
print("min: ", np.min(img0))
print("max: ", np.max(img0))
print("mean: ", np.mean(img0))
print("std: ", np.std(img0))

min:  -12.492327
max:  18.976564
mean:  0.12850438
std:  1.1625547


In [14]:
imgR, _ = resampleToPixdims(Image(img0), (1,1,1))

In [18]:
imgR.shape

(91, 109, 91)

In [22]:
img = Image(path).data[:,:,:,0]

In [23]:
img.shape

(91, 109, 91)

In [25]:
imgR = resampleToPixdims(Image(img, header=Image(path).header), (1,1,1))

In [26]:
type(imgR)

tuple

In [28]:
imgR[0].shape

(182, 218, 182)

In [29]:
imgR = resampleToPixdims(Image(img), (1,1,1))

In [30]:
imgR[0].shape

(91, 109, 91)

In [43]:
imgR, xform = resampleToPixdims(Image(img, header=Image(path).header), (1,1,1))
header = Image(imgR, header=Image(path).header, xform=xform).header

In [44]:
imgR.shape

(182, 218, 182)

In [45]:
print("min: ", np.min(img0))
print("max: ", np.max(img0))
print("mean: ", np.mean(img0))
print("std: ", np.std(img0))

min:  -12.492327
max:  18.976564
mean:  0.12850438
std:  1.1625547


In [46]:
print("min: ", np.min(imgR))
print("max: ", np.max(imgR))
print("mean: ", np.mean(imgR))
print("std: ", np.std(imgR))

min:  -12.492327
max:  18.976564
mean:  0.12850817
std:  1.1004686


In [48]:
output_nifti_image = Image(imgR, header=header, xform=xform)

In [49]:
Image(output_nifti_image).save("23679812_rsfmri_resampled")

In [36]:
imgR_i = Image(imgR)

In [39]:
imgR_i.header

In [42]:
Image(path).xform()

AttributeError: 'Image' object has no attribute 'xform'

In [ ]:
def resampleToPixdims(image, newPixdims, **kwargs):
    """Resample ``image`` so that it has the specified voxel dimensions.

    This is a wrapper around :func:`resample` - refer to its documenttion
    for details on the other arguments and the return values.

    :arg image:   :class:`.Image` to resample
    :arg pixdims: New voxel dimensions to resample ``image`` to.
    """
    newPixdims = np.array(newPixdims)
    oldShape   = np.array(image.shape)
    oldPixdims = np.array(image.pixdim)
    newShape   = oldShape * (oldPixdims / newPixdims)
    return resample(image, newShape, **kwargs)

def resample(image,
             newShape,
             import fsl.transform.affine as affineimport fsl.transform.affine as affineimport fsl.transform.affine as affineimport fsl.transform.affine as affine=None,
             dtype=None,
             order=1,
             smooth=True,
             origin=None,
             matrix=None,
             mode=None,
             cval=0):
    """Returns a copy of the data in the ``image``, resampled to the specified
    ``newShape``.

    The space that the image is resampled into can be defined in one of the
    following ways, in decreasing order of precedence:

      1. If a ``matrix`` is provided, it is applied to the voxel coordinates
         when retrieving values from the ``image``

      2. Otherwise the image is simply scaled according to the ratio calculated
         by ``image.shape / newShape``. In this case the ``origin`` argument
         may be used to adjust the alignemnt of the original and resampled
         voxel grids.

    See the ``scipy.ndimage.affine_transform`` function for more details,
    particularly on the ``order``, ``matrix``, ``mode`` and
    ``cval`` arguments.

    .. note:: If a custom resampling ``matrix`` is specified, the adjusted
              voxel-to-world afffine cannot be calculated by this function,
              so ``None`` will be returned instead.

    :arg image:    :class:`.Image` object to resample

    :arg newShape: Desired shape. May containg floating point values, in which
                   case the resampled image will have shape
                   ``round(newShape)``, but the voxel sizes will have scales
                   ``self.shape / newShape`` (unless ``matrix`` is specified).

    :arg sliceobj: Slice into this ``Image``. If ``None``, the whole
                   image is resampled, and it is assumed that it has the
                   same number of dimensions as  ``newShape``. A
                   :exc:`ValueError` is raised if this is not the case.

    :arg dtype:    ``numpy`` data type of the resampled data. If ``None``,
                   the :meth:`dtype` of this ``Image`` is used.

    :arg order:    Spline interpolation order, passed through to the
                   ``scipy.ndimage.affine_transform`` function - ``0``
                   corresponds to nearest neighbour interpolation, ``1``
                   (the default) to linear interpolation, and ``3`` to
                   cubic interpolation.

    :arg smooth:   If ``True`` (the default), the data is smoothed before
                   being resampled, but only along axes which are being
                   down-sampled (i.e. where ``newShape[i] < self.shape[i]``).

    :arg origin:   ``'centre'`` (the default) or ``'corner'``. ``'centre'``
                   resamples the image such that the centre of the corner
                   voxels of this image and the resampled data are
                   aligned. ``'corner'`` resamples the image such that
                   the corner of the corner voxels are aligned (and
                   therefore the voxel grids are aligned).
                   Ignored if ``offset`` or ``matrix`` is specified.

    :arg matrix:   Arbitrary affine transformation matrix to apply to the
                   voxel coordinates of ``image`` when resampling.

    :arg mode:     How to handle regions which are outside of the image FOV.
                   Defaults to `''nearest'``.

    :arg cval:     Constant value to use when ``mode='constant'``.

    :returns: A tuple containing:

               - A ``numpy`` array of shape ``newShape``, containing
                 an interpolated copy of the data in this ``Image``.

               - A ``numpy`` array of shape ``(4, 4)``, containing the
                 adjusted voxel-to-world transformation for the spatial
                 dimensions of the resampled data, or ``None`` if a ``matrix``
                 was provided.
    """

    if sliceobj is None:     sliceobj = slice(None)
    if dtype    is None:     dtype    = image.dtype
    if origin   is None:     origin   = 'centre'
    if mode     is None:     mode     = 'nearest'
    if origin   == 'center': origin   = 'centre'

    ownMatrix = matrix is None

    if origin not in ('centre', 'corner'):
        raise ValueError('Invalid value for origin: {}'.format(origin))

    data = np.array(image[sliceobj], dtype=dtype, copy=False)

    if len(data.shape) != len(newShape):
        raise ValueError('Data dimensions do not match new shape: '
                         'len({}) != len({})'.format(data.shape, newShape))

    # If matrix not provided, calculate
    # a scaling/offset matrix from the
    # old/new shape ratio and the origin
    # setting.
    if matrix is None:
        matrix = affine.rescale(data.shape, newShape, origin)

    # same shape and identity matrix? the
    # image doesn't need to be resampled
    if np.all(np.isclose(data.shape, newShape)) and \
       np.all(np.isclose(matrix, np.eye(len(newShape) + 1))):
        return data, image.voxToWorldMat

    newShape = np.array(np.round(newShape), dtype=int)

    # Apply smoothing if requested,
    # and if not using nn interp
    if order > 0 and smooth:
        data = applySmoothing(data, matrix, newShape)

    # Do the resample thing
    data = ndimage.affine_transform(data,
                                    matrix,
                                    output_shape=newShape,
                                    order=order,
                                    mode=mode,
                                    cval=cval)

    # Construct an affine transform which
    # puts the resampled image into the
    # same world coordinate system as this
    # image. We may be working with >3D data,
    # so here we discard the non-spatial
    # parts of the resampling matrix
    if matrix.shape != (4, 4):
        rotmat         = matrix[:3, :3]
        offsets        = matrix[:3, -1]
        matrix         = np.eye(4)
        matrix[:3, :3] = rotmat
        matrix[:3, -1] = offsets

    # We can only adjust the v2w affine if
    # the input space and resampling space
    # are aligned (e.g. if we're just
    # resampling to different dimensions).
    # We can't assume this when a custom
    # matrix is specified, so we just give
    # up and return None.
    if ownMatrix: matrix = affine.concat(image.voxToWorldMat, matrix)
    else:         matrix = None

    return data, matrix



def applySmoothing(data, matrix, newShape):
    """Called by the :func:`resample` function.

    If interpolating and smoothing, we apply a gaussian filter along axes with
    a resampling ratio greater than 1.1. We do this so that interpolation has
    an effect when down-sampling to a resolution where the voxel centres are
    aligned (as otherwise any interpolation regime will be equivalent to
    nearest neighbour). This more-or-less mimics the behaviour of FLIRT.

    See the ``scipy.ndimage.gaussian_filter`` function for more details.

    :arg data:     Data to be smoothed.
    :arg matrix:   Affine matrix to be used during resampling. The voxel
                   scaling factors are extracted from this.
    :arg newShape: Shape the data is to be resampled into.
    :returns:      A smoothed copy of ``data``.
    """

    ratio = affine.decompose(matrix[:3, :3])[0]

    if len(newShape) > 3:
        ratio = np.concatenate((
            ratio,
            [float(o) / float(s)
             for o, s in zip(data.shape[3:], newShape[3:])]))

    sigma                = np.array(ratio)
    sigma[ratio <  1.1]  = 0
    sigma[ratio >= 1.1] *= 0.425

    return ndimage.gaussian_filter(data, sigma)

In [3]:
# F, xform = resampleToPixdims(Image(F, header=Image(pathF).header), (1,1,1))

import os
import pickle
import numpy as np
import configparser
import pandas as pd
from fsl.data.image import Image
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import numpy as np
from fsl.data.image import Image
from fsl.utils.image.resample import resampleToPixdims
from fsl.utils.image.roi import roi
import torch
import nibabel as nib

In [4]:
pathF = "23679812_rsfmri_original"
F = Image(pathF).data[:,:,:,0]

In [63]:
def resampleToPixdims(image, newPixdims):

    newPixdims = np.array(newPixdims)
    oldShape   = np.array(image.shape)
    oldPixdims = np.array(image.pixdim)
    newShape   = oldShape * (oldPixdims / newPixdims)
    
#     print(newPixdims, oldShape, oldPixdims, newShape)
#     print(type(newPixdims), type(oldShape), type(oldPixdims), type(newShape))
#     print(image.dtype)
    return resample(image, oldShape, newShape)

In [64]:
# resampleToPixdims(Image(F, header=Image(pathF).header), (1,1,1))

In [65]:
def resampleToPixdims2(image, oldPixdims, newPixdims):

    newPixdims = np.array(newPixdims)
    oldShape   = np.array(image.shape)
    oldPixdims = np.array(oldPixdims)
    newShape   = oldShape * (oldPixdims / newPixdims)
    
#     print(newPixdims, oldShape, oldPixdims, newShape)
#     print(type(newPixdims), type(oldShape), type(oldPixdims), type(newShape))
#     print(image.dtype)
    return resample(image, oldShape, newShape)

In [92]:
pathF = "23679812_rsfmri_original.nii.gz"
F2 = np.array(nib.load(pathF).dataobj)[:,:,:,0]

In [28]:
resampleToPixdims2(F2, (2,2,2), (1,1,1))

float32


In [117]:
import fsl.transform.affine as affine

def rescale(oldShape, newShape, origin):

    ndim     = len(oldShape)
    ratio = oldShape / newShape
    scale = np.diag(ratio)
    offset = [0] * ndim
    xform               = np.eye(ndim + 1)
    xform[:ndim, :ndim] = scale
    xform[:ndim, -1]    = offset

    return xform

def resample(image,
             oldShape,
             newShape,
             sliceobj=None,
             dtype=None,
             order=1,
             smooth=True,
             origin=None,
             matrix=None,
             mode=None,
             cval=0):

    sliceobj = slice(None)
    dtype    = image.dtype
    origin   = 'centre'
    mode     = 'nearest'

    ownMatrix = matrix is None
    data = np.array(image[sliceobj], dtype=dtype, copy=False)

    if matrix is None:
        matrix = rescale(oldShape, newShape, origin)
        
    newShape = np.array(np.round(newShape), dtype=int)

    data = ndimage.affine_transform(data,
                                    matrix,
                                    output_shape=newShape,
                                    order=order,
                                    mode=mode,
                                    cval=cval)

    return data

In [118]:
out1 = resampleToPixdims(Image(F, header=Image(pathF).header), (1,1,1))

In [119]:
out2 = resampleToPixdims2(F2, (2,2,2), (1,1,1))

In [107]:
Image(out1).save('F1.nii.gz')
Image(out2).save('F2.nii.gz')

In [87]:
type(F1)

NoneType

In [103]:
from scipy.ndimage import gaussian_filter
a = np.arange(50, step=2).reshape((5,5))
print(a)
print(gaussian_filter(a, sigma=0))

[[ 0  2  4  6  8]
 [10 12 14 16 18]
 [20 22 24 26 28]
 [30 32 34 36 38]
 [40 42 44 46 48]]
[[ 0  2  4  6  8]
 [10 12 14 16 18]
 [20 22 24 26 28]
 [30 32 34 36 38]
 [40 42 44 46 48]]


In [111]:
out2.shape

(91, 109, 91)

In [114]:
upsampled_array = np.kron(out2, np.ones((2, 2, 2)))

In [115]:
upsampled_array.shape

(182, 218, 182)

In [116]:
Image(upsampled_array).save('F3.nii.gz')

In [123]:
import time

start_time = time.time()
iterations = 200
for _ in range(iterations):
    F = Image(pathF).data[:,:,:,0]
    F = resampleToPixdims2(F, (2,2,2), (1,1,1))
#     F = roi(Image(F, header=Image(pathF).header, xform=xform),((10,170),(12,204),(0,160))).data
#     F = F / 5.678573626611138
    
#     x_shift, y_shift, z_shift = np.random.randint(-2,3,3)
#     F = np.roll(F,x_shift,axis=0)
#     F = np.roll(F,y_shift,axis=1)
#     F = np.roll(F,z_shift,axis=2)
#     if z_shift < 0:
#         X_volume[:,:,z_shift:] = 0
    
print("--- %s seconds ---" % (time.time() - start_time))
print("--- %s sec/ite ---" % ((time.time() - start_time)/iterations))

--- 177.25169277191162 seconds ---
--- 0.8862591087818146 sec/ite ---


In [125]:
import time

start_time = time.time()
iterations = 200
for _ in range(iterations):
    F = np.array(nib.load(pathF).dataobj)[:,:,:,0]
    F = resampleToPixdims2(F, (2,2,2), (1,1,1))
#     F = roi(Image(F, header=Image(pathF).header, xform=xform),((10,170),(12,204),(0,160))).data
#     F = F / 5.678573626611138
    
#     x_shift, y_shift, z_shift = np.random.randint(-2,3,3)
#     F = np.roll(F,x_shift,axis=0)
#     F = np.roll(F,y_shift,axis=1)
#     F = np.roll(F,z_shift,axis=2)
#     if z_shift < 0:
#         X_volume[:,:,z_shift:] = 0
    
print("--- %s seconds ---" % (time.time() - start_time))
print("--- %s sec/ite ---" % ((time.time() - start_time)/iterations))

--- 160.90542793273926 seconds ---
--- 0.804528136253357 sec/ite ---


In [126]:
def resampleToPixdims3(image, oldPixdims, newPixdims):

    newPixdims = np.array(newPixdims)
    oldShape   = np.array(image.shape)
    oldPixdims = np.array(oldPixdims)
    newShape   = oldShape * (oldPixdims / newPixdims)
    
    return resample2(image, oldShape, newShape)

def resample2(image,
             oldShape,
             newShape,
             sliceobj=None,
             dtype=None,
             order=1,
             smooth=True,
             origin=None,
             matrix=None,
             mode=None,
             cval=0):

    sliceobj = slice(None)
    dtype    = image.dtype
    origin   = 'centre'
    mode     = 'nearest'

    ownMatrix = matrix is None
    data = np.array(image[sliceobj], dtype=dtype, copy=False)

    if matrix is None:
        matrix = rescale(oldShape, newShape, origin)
        
    newShape = np.array(np.round(newShape), dtype=int)

    data = np.kron(data, np.ones((2, 2, 2)))

    return data

In [127]:
import time

start_time = time.time()
iterations = 200
for _ in range(iterations):
    F = np.array(nib.load(pathF).dataobj)[:,:,:,0]
    F = resampleToPixdims3(F, (2,2,2), (1,1,1))
#     F = roi(Image(F, header=Image(pathF).header, xform=xform),((10,170),(12,204),(0,160))).data
#     F = F / 5.678573626611138
    
#     x_shift, y_shift, z_shift = np.random.randint(-2,3,3)
#     F = np.roll(F,x_shift,axis=0)
#     F = np.roll(F,y_shift,axis=1)
#     F = np.roll(F,z_shift,axis=2)
#     if z_shift < 0:
#         X_volume[:,:,z_shift:] = 0
    
print("--- %s seconds ---" % (time.time() - start_time))
print("--- %s sec/ite ---" % ((time.time() - start_time)/iterations))

--- 74.6341860294342 seconds ---
--- 0.37317193508148194 sec/ite ---


In [245]:
import time

m = torch.nn.Upsample(scale_factor=2, mode='trilinear', align_corners=True)

start_time = time.time()
iterations = 200
for _ in range(iterations):
    F = np.array(nib.load(pathF).dataobj)[:,:,:,0]
        
    sp = (1,1)+F.shape
    F = F.reshape(sp)
    F = torch.tensor(F, dtype=torch.float32)
    F = m(F)
# X_volume = torch.squeeze(X_volume)
# X_volume = X_volume.detach().cpu().numpy()
# X_volume.squeeze()
# Image(X_volume).save('F_upsampled_align_corners.nii.gz')
    
#     F = roi(Image(F, header=Image(pathF).header, xform=xform),((10,170),(12,204),(0,160))).data
#     F = F / 5.678573626611138
    
#     x_shift, y_shift, z_shift = np.random.randint(-2,3,3)
#     F = np.roll(F,x_shift,axis=0)
#     F = np.roll(F,y_shift,axis=1)
#     F = np.roll(F,z_shift,axis=2)
#     if z_shift < 0:
#         X_volume[:,:,z_shift:] = 0
    
print("--- %s seconds ---" % (time.time() - start_time))
print("--- %s sec/ite ---" % ((time.time() - start_time)/iterations))

--- 69.74571990966797 seconds ---
--- 0.3487356400489807 sec/ite ---


In [247]:
import time

start_time = time.time()
iterations = 200
for _ in range(iterations):
    F = np.array(nib.load(pathF).dataobj)[:,:,:,0]
    F = resampleToPixdims3(F, (2,2,2), (1,1,1))


    F = F[10:170, 12:204, 0:160]
    F = F / 5.678573626611138
    
    x_shift, y_shift, z_shift = np.random.randint(-2,3,3)
    F = np.roll(F,x_shift,axis=0)
    F = np.roll(F,y_shift,axis=1)
    F = np.roll(F,z_shift,axis=2)
    if z_shift < 0:
        F[:,:,z_shift:] = 0
    
print("--- %s seconds ---" % (time.time() - start_time))
print("--- %s sec/ite ---" % ((time.time() - start_time)/iterations))

--- 84.14250898361206 seconds ---
--- 0.4207188451290131 sec/ite ---


In [248]:
import time

m = torch.nn.Upsample(scale_factor=2, mode='trilinear', align_corners=True)

start_time = time.time()
iterations = 200
for _ in range(iterations):
    F = np.array(nib.load(pathF).dataobj)[:,:,:,0]
        
    sp = (1,1)+F.shape
    F = F.reshape(sp)
    F = torch.tensor(F, dtype=torch.float32)
    F = m(F)
    F = torch.squeeze(F)
    F = F.detach().cpu().numpy()
    
    F = F[10:170, 12:204, 0:160]
    F = F / 5.678573626611138
    
    x_shift, y_shift, z_shift = np.random.randint(-2,3,3)
    F = np.roll(F,x_shift,axis=0)
    F = np.roll(F,y_shift,axis=1)
    F = np.roll(F,z_shift,axis=2)
    if z_shift < 0:
        X_volume[:,:,z_shift:] = 0
    
print("--- %s seconds ---" % (time.time() - start_time))
print("--- %s sec/ite ---" % ((time.time() - start_time)/iterations))

--- 68.52946496009827 seconds ---
--- 0.3426482951641083 sec/ite ---


In [129]:
import torch

In [137]:
a = torch.arange(1,5,dtype=torch.float32).view(1,1,2,2)
print(a)
m = torch.nn.Upsample(scale_factor=1, mode='nearest')
print(m(a), (a==m(a)).all())
m = torch.nn.Upsample(scale_factor=2, mode='nearest')
print(m(a))

tensor([[[[1., 2.],
          [3., 4.]]]])
tensor([[[[1., 2.],
          [3., 4.]]]]) tensor(True)
tensor([[[[1., 1., 2., 2.],
          [1., 1., 2., 2.],
          [3., 3., 4., 4.],
          [3., 3., 4., 4.]]]])


In [249]:
pathT1 = 'T1s/T1_brain_to_MNI.nii.gz'
pathF = "23679812_rsfmri_original.nii.gz"

In [250]:
X_volume = np.array(nib.load(pathT1).dataobj)
X_volume = X_volume[10:170, 12:204, 0:160]
X_volume = X_volume / X_volume.mean()
X_volume = X_volume / 5.678573626611138

In [251]:
Image(X_volume).save('T1_not_upsampled.nii.gz')

In [252]:
m = torch.nn.Upsample(scale_factor=1, mode='trilinear', align_corners=True)

In [253]:
sp = (1,1)+X_volume.shape
X_volume = X_volume.reshape(sp)
X_volume = torch.tensor(X_volume, dtype=torch.float32)
X_volume = m(X_volume)
X_volume = torch.squeeze(X_volume)
X_volume = X_volume.detach().cpu().numpy()
X_volume.squeeze()
Image(X_volume).save('T1_upsampled_corners.nii.gz')

In [173]:
a = torch.rand(2,1,3,3,3)

In [177]:
print(a)
m = torch.nn.Upsample(scale_factor=1, mode='trilinear', align_corners=False)
print((m(a)==a).all())

tensor([[[[[0.9189, 0.8165, 0.9447],
           [0.1760, 0.4569, 0.5310],
           [0.7899, 0.9176, 0.6699]],

          [[0.0687, 0.8723, 0.8643],
           [0.8963, 0.0224, 0.3680],
           [0.7319, 0.8368, 0.0723]],

          [[0.3943, 0.0892, 0.2589],
           [0.7264, 0.0710, 0.1701],
           [0.9113, 0.7552, 0.7415]]]],



        [[[[0.2967, 0.4662, 0.2450],
           [0.7177, 0.6322, 0.9085],
           [0.5695, 0.8751, 0.9957]],

          [[0.0314, 0.5051, 0.7263],
           [0.1331, 0.3135, 0.8340],
           [0.0552, 0.9283, 0.3625]],

          [[0.1812, 0.4284, 0.1958],
           [0.1006, 0.2259, 0.9975],
           [0.0578, 0.1856, 0.5478]]]]])
tensor(True)


In [240]:
X_volume = np.array(nib.load(pathF).dataobj)[:,:,:,0]
# X_volume = X_volume[5:85, 6:102, 0:80]
# X_volume = X_volume / X_volume.mean()
# X_volume = X_volume / 5.678573626611138

In [241]:
Image(X_volume).save('F_not_upsampled.nii.gz')

In [242]:
X_volume.shape

(91, 109, 91)

In [243]:
m = torch.nn.Upsample(scale_factor=2, mode='trilinear', align_corners=True)
# m = torch.nn.Upsample(scale_factor=2, mode='nearest')

In [244]:
sp = (1,1)+X_volume.shape
X_volume = X_volume.reshape(sp)
X_volume = torch.tensor(X_volume, dtype=torch.float32)
X_volume = m(X_volume)
X_volume = torch.squeeze(X_volume)
X_volume = X_volume.detach().cpu().numpy()
X_volume.squeeze()
Image(X_volume).save('F_upsampled_align_corners.nii.gz')

In [226]:
X_volume.shape

(182, 218, 182)

In [191]:
a = torch.rand(2,1,3,3,3)
print(a)
m = torch.nn.Upsample(scale_factor=2, mode='nearest ', align_corners=False)
print(m(a).shape)
print(m(a))

tensor([[[[[0.4215, 0.9487, 0.6148],
           [0.9554, 0.1768, 0.2370],
           [0.6592, 0.3822, 0.0218]],

          [[0.3086, 0.1507, 0.1317],
           [0.4160, 0.3796, 0.8977],
           [0.8321, 0.4448, 0.7884]],

          [[0.1443, 0.1440, 0.8496],
           [0.4992, 0.7481, 0.8816],
           [0.1490, 0.9836, 0.5094]]]],



        [[[[0.8054, 0.4583, 0.9790],
           [0.7815, 0.3667, 0.7266],
           [0.6912, 0.9054, 0.8204]],

          [[0.8273, 0.0209, 0.8125],
           [0.9156, 0.4734, 0.7948],
           [0.3005, 0.1378, 0.0239]],

          [[0.0351, 0.9434, 0.1124],
           [0.0092, 0.5157, 0.3472],
           [0.6070, 0.0259, 0.3083]]]]])
torch.Size([2, 1, 6, 6, 6])
tensor([[[[[0.4215, 0.5533, 0.8169, 0.8652, 0.6983, 0.6148],
           [0.5550, 0.6052, 0.7055, 0.6969, 0.5792, 0.5204],
           [0.8219, 0.7089, 0.4828, 0.3602, 0.3410, 0.3315],
           [0.8814, 0.7181, 0.3915, 0.2169, 0.1944, 0.1832],
           [0.7333, 0.6327, 0.4314, 0.2670, 